In [230]:
import numpy as np
import pandas as pd

In [231]:
df = pd.read_csv('/content/IMDB-Movie-Data.csv')

In [232]:
df.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [233]:
df.columns

Index(['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore'],
      dtype='object')

In [234]:
df.isnull().sum()

Rank                    0
Title                   0
Genre                   0
Description             0
Director                0
Actors                  0
Year                    0
Runtime (Minutes)       0
Rating                  0
Votes                   0
Revenue (Millions)    128
Metascore              64
dtype: int64

Simply dropping column instaed of replacing null values because thise columns are not important anyways.

In [235]:
df = df.drop(['Revenue (Millions)', 'Metascore'], axis = 1)

Columns after dropping.

In [236]:
df.columns

Index(['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes'],
      dtype='object')

There are total of 1000 movies.

In [237]:
df.shape

(1000, 10)

Taking all the columns that are important.We will be using data of these columns to create our recommendations.

In [238]:
imp_columns = ['Title', 'Genre', 'Description', 'Director', 'Actors']

In [239]:
df[imp_columns].head(3)

,Title,Genre,Description,Director,Actors
0,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S..."
1,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa..."
2,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar..."


This combines the text from all these important columns into a string then we craete a list of those 

In [240]:
def combine_features(data, imp_columns):
  combined_features = []
  for i in range(0, data.shape[0]):
    result = ''
    for feature in imp_columns:
      result = result + ' ' + str(data[feature][i])  # if result = 'split'  ..... result = 'split' + ' ' + str(data[Genre][2]) where 2 is row number
    combined_features.append(result) # appending entire row containing 'Title', 'Genre', 'Description', 'Director', 'Actors' in a list element
  return combined_features           # every list element represents a movie

Putting it in a new column.

In [241]:
df['combined_features'] = combine_features(df, imp_columns)

In [242]:
df.tail(1)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,combined_features
999,1000,Nine Lives,"Comedy,Family,Fantasy",A stuffy businessman finds himself trapped ins...,Barry Sonnenfeld,"Kevin Spacey, Jennifer Garner, Robbie Amell,Ch...",2016,87,5.3,12435,"Nine Lives Comedy,Family,Fantasy A stuffy bus..."


In [243]:
df.iloc[999]['combined_features']

" Nine Lives Comedy,Family,Fantasy A stuffy businessman finds himself trapped inside the body of his family's cat. Barry Sonnenfeld Kevin Spacey, Jennifer Garner, Robbie Amell,Cheryl Hines"

Count vectorizer now converts entire texts to vectors.

In [246]:
from sklearn.feature_extraction.text import CountVectorizer
cm = CountVectorizer()
cm.fit(df['combined_features'])
cm.vocabulary_

{'guardians': 3677,
 'of': 6015,
 'the': 8332,
 'galaxy': 3391,
 'action': 178,
 'adventure': 225,
 'sci': 7365,
 'fi': 3095,
 'group': 3663,
 'intergalactic': 4299,
 'criminals': 2004,
 'are': 525,
 'forced': 3222,
 'to': 8445,
 'work': 9240,
 'together': 8453,
 'stop': 7968,
 'fanatical': 3011,
 'warrior': 9015,
 'from': 3325,
 'taking': 8186,
 'control': 1878,
 'universe': 8747,
 'james': 4425,
 'gunn': 3694,
 'chris': 1599,
 'pratt': 6548,
 'vin': 8904,
 'diesel': 2367,
 'bradley': 1152,
 'cooper': 1894,
 'zoe': 9336,
 'saldana': 7259,
 'prometheus': 6632,
 'mystery': 5781,
 'following': 3208,
 'clues': 1692,
 'origin': 6105,
 'mankind': 5266,
 'team': 8246,
 'finds': 3138,
 'structure': 8014,
 'on': 6061,
 'distant': 2422,
 'moon': 5665,
 'but': 1309,
 'they': 8357,
 'soon': 7787,
 'realize': 6833,
 'not': 5955,
 'alone': 350,
 'ridley': 7058,
 'scott': 7381,
 'noomi': 5942,
 'rapace': 6796,
 'logan': 5057,
 'marshall': 5323,
 'green': 3627,
 'michael': 5514,
 'fassbender': 3034,


In [247]:
cm = cm.transform(df['combined_features'])

In [248]:
cm

<1000x9359 sparse matrix of type '<class 'numpy.int64'>'
	with 38547 stored elements in Compressed Sparse Row format>

Cosine similarity measures the similarity between vectors based on cosine angle between them. The more the value the better.
If cos(theta) = 1, overlaps .i.e same vectors

We will be plotting pairwise cosine similarity; 1000 movies vs 1000 movies
If 1, means same movie.

In [147]:
from sklearn.metrics.pairwise import cosine_similarity
cs = cosine_similarity(cm)
print(cs)

[[1.         0.25692801 0.21021383 ... 0.2306328  0.25391836 0.13543224]
 [0.25692801 1.         0.15384615 ... 0.10127394 0.1672484  0.08920516]
 [0.21021383 0.15384615 1.         ... 0.10127394 0.13937367 0.0594701 ]
 ...
 [0.2306328  0.10127394 0.10127394 ... 1.         0.13762047 0.11744404]
 [0.25391836 0.1672484  0.13937367 ... 0.13762047 1.         0.09697623]
 [0.13543224 0.08920516 0.0594701  ... 0.11744404 0.09697623 1.        ]]


Here each row and column represnt a movie, and the correlation between each is given, where it is 1, it means they are the same movie. It'll be 1 across diagonal.

In [148]:
cs.shape

(1000, 1000)

Picking any random movie, say user likes this movie say 'Fury'.

In [214]:
movie_title = 'Fury'

Getting index of the movie.

In [215]:
movie_id = df[df['Title'] == movie_title].index.values

In [216]:
movie_id

array([240])

Selecting entire cosine similarity row of 'Fury'. As movie id is 240 as can be seen from above, obviously selected_movie_cs[0][240] = 1

In [218]:
selected_movie_cs = cs[movie_id]

In [219]:
selected_movie_cs.shape

(1, 1000)

In [220]:
type(selected_movie_cs)

numpy.ndarray

As selected_movie_cs is an numpy array. We will sort them in ascending order. Hence movies with max cosine similarity will be at end. So what argsort does is it sorts indexes instead. Hence this sort_score will contain indexes of movies. As the sorting is ascending then last element will be the index of movie 'Fury' as cs was 1 and so on

In [221]:
sort_scores = np.argsort(selected_movie_cs)

In [222]:
sort_scores.shape

(1, 1000)

Now in the below cell we are selecting 10 values from last ignoring the last one as it is the movie itself.

In [223]:
recommendation = sort_scores[:, sort_scores.shape[0]-12:-1]

So the movie with index 210 is the most relatable to 'Fury'.

In [224]:
recommendation

array([[920, 700, 188, 530, 301, 810, 460,  71, 359, 210]])

In [225]:
type(recommendation.shape[1])

int

Now getting the Titles of the movie, and appending them into a list.

In [226]:
movies = []
for i in range(0, recommendation.shape[1]):
  id = recommendation[0][i]
  movies.append(df.iloc[id]['Title'])

Reversing the list, to make the most relatable movie as first cause cs was sorted ascending.

In [227]:
movies = movies[::-1]

List of 10 most recommended movies.

In [228]:
movies

['San Andreas',
 'About Time',
 'Allied',
 'The Town',
 'Turbo Kid',
 'Lone Survivor',
 'A Good Year',
 'Drive',
 'Funny Games',
 'Centurion']